In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [31]:
# Load Dataframes
INPUT_FOLDER = "C:\\Users\\rcpat\\Desktop\\personal-projects\\MarchMadness\\2023\\Data\\"
reg_season = pd.read_csv(INPUT_FOLDER + "MRegularSeasonCompactResults.csv").drop(['NumOT','WLoc'], axis = 1)
tournament = pd.read_csv(INPUT_FOLDER + "MNCAATourneyCompactResults.csv").drop(['NumOT', 'WLoc'], axis=1)
ordinals1 = pd.read_csv(INPUT_FOLDER + "MMasseyOrdinals.csv")
ordinals2 = pd.read_csv(INPUT_FOLDER + "MMasseyOrdinals_2023_133_only_65systems.csv")
seeds = pd.read_csv(INPUT_FOLDER + 'MNCAATourneySeeds.csv')
teams = pd.read_csv(INPUT_FOLDER + "MTeams.csv")
team_spellings = pd.read_csv(INPUT_FOLDER + "MTeamSpellings.csv", encoding='cp1252')
FTE = pd.read_csv(INPUT_FOLDER + "538ratingsMen.csv").drop(['TeamName'], axis = 1)

ordinals = pd.concat([ordinals1, ordinals2])

seeds['Seed'] = seeds['Seed'].str.extract('(\d+)').astype('int')
tournament['Season'] = tournament['Season'].astype('int')

In [32]:
# Reg Season
reg_season = reg_season[reg_season['Season'] > 2015]

reg_season['ScoreDiff'] = reg_season['WScore'] - reg_season['LScore']

wins = reg_season.groupby(['Season','WTeamID']).\
count().\
reset_index()[['Season', 'WTeamID', 'DayNum']].\
rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})

losses = reg_season.groupby(['Season','LTeamID']).\
count().\
reset_index()[['Season', 'LTeamID', 'DayNum']].\
rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})

gap_win = reg_season.groupby(['Season', 'WTeamID']).mean().reset_index()[['Season','WTeamID','ScoreDiff']]\
.rename(columns={"ScoreDiff": "GapWins", "WTeamID": "TeamID"})
gap_loss = reg_season.groupby(['Season', 'LTeamID']).mean().reset_index()[['Season','LTeamID','ScoreDiff']]\
.rename(columns={"ScoreDiff": "GapLosses", "LTeamID": "TeamID"})

# Wins in last 10 games
#wins_l10 = reg_season.groupby(['Season', 'WTeamID'])['DayNum'].nlargest(10)

# Losses in last 10 games
#losses_l10 = 


df_features_season_w = reg_season.\
groupby(['Season', 'WTeamID']).\
count().\
reset_index()[['Season', 'WTeamID']].\
rename(columns={"WTeamID": "TeamID"})

df_features_season_l = reg_season.\
groupby(['Season', 'LTeamID']).\
count().reset_index()[['Season', 'LTeamID']].\
rename(columns={"LTeamID": "TeamID"})

df_features_season = pd.concat([df_features_season_w, df_features_season_l], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

df_features_season = df_features_season.merge(wins, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(losses, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')


df_features_season.fillna(0, inplace=True)

df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

df_features_season.drop(['NumWins', 'NumLosses', 'GapWins', 'GapLosses'], axis=1, inplace=True)

df_features_season = df_features_season.merge(FTE, how = 'inner', on = ['Season','TeamID'])

df_features_season.head()

C:\Users\rcpat\AppData\Local\Temp\ipykernel_14896\1806530786.py:39: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.



,Season,TeamID,WinRatio,GapAvg,538rating
0,2016,1112,0.757576,12.212121,89.04
1,2016,1114,0.870968,9.935484,78.90
2,2016,1122,0.484848,-2.363636,68.83
3,2016,1124,0.656250,6.687500,85.47
4,2016,1138,0.575758,0.666667,75.66


In [59]:
# Get winning Pct in last 10 games of reg season

tempW = pd.DataFrame.from_dict({'Season':reg_season['Season'], 'TeamID':reg_season['WTeamID'],\
                                   'DayNum':reg_season['DayNum'], 'Result': 1}).reset_index(drop = True)

tempL = pd.DataFrame.from_dict({'Season':reg_season['Season'], 'TeamID':reg_season['LTeamID'],\
                                   'DayNum':reg_season['DayNum'], 'Result': 0}).reset_index(drop = True)

team_results = pd.concat([tempW,tempL])

l10 = team_results.groupby(['Season','TeamID'])['DayNum'].nlargest(10).reset_index().drop(['level_2'], axis = 1)

l10_results = l10.merge(team_results, how = 'left', on = ['Season', 'TeamID', 'DayNum']).groupby(['Season','TeamID']).mean()

l10_results = l10_results.reset_index().drop(['DayNum'], axis = 1).rename(columns={'Result':'WinRatio_L10'})

df_features_season = df_features_season.merge(l10_results, how = 'left', left_on = ['Season','TeamID'], right_on=['Season','TeamID'])

df_features_season['WinRatio_SeasonL10_Diff'] = df_features_season['WinRatio'] - df_features_season['WinRatio_L10']

df_features_season.head()

,Season,TeamID,WinRatio,GapAvg,538rating,WinRatio_L10,WinRatio_SeasonL10_Diff
0,2016,1112,0.757576,12.212121,89.04,0.7,0.057576
1,2016,1114,0.870968,9.935484,78.90,0.8,0.070968
2,2016,1122,0.484848,-2.363636,68.83,0.8,-0.315152
3,2016,1124,0.656250,6.687500,85.47,0.5,0.156250
4,2016,1138,0.575758,0.666667,75.66,0.6,-0.024242


In [60]:
# Add KenPom
ordinals_df = ordinals[ordinals['Season'] >= 2016]
POM = ordinals_df[ordinals_df['SystemName'] == 'POM'].\
groupby(['Season','TeamID']).\
max().\
drop(['RankingDayNum','SystemName'], axis = 1).\
reset_index().\
rename(columns = {'OrdinalRank':'KenPom'})

df_features_season = df_features_season.merge(POM, how = 'left', left_on = ['Season','TeamID'], right_on=['Season','TeamID'])

# Add KenPom change in last 4 weeks


df_features_season.head()

,Season,TeamID,WinRatio,GapAvg,538rating,WinRatio_L10,WinRatio_SeasonL10_Diff,KenPom
0,2016,1112,0.757576,12.212121,89.04,0.7,0.057576,19
1,2016,1114,0.870968,9.935484,78.90,0.8,0.070968,213
2,2016,1122,0.484848,-2.363636,68.83,0.8,-0.315152,287
3,2016,1124,0.656250,6.687500,85.47,0.5,0.156250,38
4,2016,1138,0.575758,0.666667,75.66,0.6,-0.024242,179


In [61]:
# Add Seeds
df_features_season = pd.merge(
    df_features_season, 
    seeds, 
    how='left', 
    on=['Season', 'TeamID']
)

df_features_season.head()

,Season,TeamID,WinRatio,GapAvg,538rating,WinRatio_L10,WinRatio_SeasonL10_Diff,KenPom,Seed
0,2016,1112,0.757576,12.212121,89.04,0.7,0.057576,19,6.0
1,2016,1114,0.870968,9.935484,78.90,0.8,0.070968,213,12.0
2,2016,1122,0.484848,-2.363636,68.83,0.8,-0.315152,287,16.0
3,2016,1124,0.656250,6.687500,85.47,0.5,0.156250,38,5.0
4,2016,1138,0.575758,0.666667,75.66,0.6,-0.024242,179,14.0


In [62]:
# Tournament Results
tourney = tournament[tournament['Season'] > 2015].reset_index(drop = True)

tourney.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,2016,134,1195,96,1192,65
1,2016,134,1455,70,1435,50
2,2016,135,1221,59,1380,55
3,2016,135,1276,67,1409,62
4,2016,136,1114,85,1345,83


In [63]:
# Season Stats
df = pd.merge(
    tourney,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'GapWins': 'GapWinsW',
    'GapLosses': 'GapLossesW',
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW',
    '538rating': '538RatingW',
    'WinRatio_L10': 'WinRatio_L10W',
    'WinRatio_SeasonL10_Diff': 'WinRatio_SeasonL10_DiffW',
    'KenPom': 'KenPomW',
    'Seed': 'SeedW'
}).drop(columns='TeamID', axis=1)

df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'GapWins': 'GapWinsL',
    'GapLosses': 'GapLossesL',
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL',
    '538rating': '538RatingL',
    'WinRatio_L10': 'WinRatio_L10L',
    'WinRatio_SeasonL10_Diff': 'WinRatio_SeasonL10_DiffL',
    'KenPom': 'KenPomL',
    'Seed': 'SeedL'
}).drop(columns='TeamID', axis=1)

df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WinRatioW,GapAvgW,538RatingW,WinRatio_L10W,WinRatio_SeasonL10_DiffW,KenPomW,SeedW,WinRatioL,GapAvgL,538RatingL,WinRatio_L10L,WinRatio_SeasonL10_DiffL,KenPomL,SeedL
0,2016,134,1195,96,1192,65,0.551724,2.724138,71.41,0.7,-0.148276,264,16.0,0.548387,-1.677419,66.72,0.6,-0.051613,321,16.0
1,2016,134,1455,70,1435,50,0.741935,13.709677,86.59,0.7,0.041935,44,11.0,0.593750,9.406250,85.59,0.6,-0.006250,38,11.0
2,2016,135,1221,59,1380,55,0.424242,-4.333333,66.85,0.5,-0.075758,324,16.0,0.612903,1.870968,67.96,0.6,0.012903,263,16.0
3,2016,135,1276,67,1409,62,0.636364,6.242424,79.57,0.5,0.136364,56,11.0,0.645161,4.322581,79.93,0.6,0.045161,78,11.0
4,2016,136,1114,85,1345,83,0.870968,9.935484,78.90,0.8,0.070968,213,12.0,0.764706,13.147059,88.68,0.7,0.064706,19,5.0


In [64]:
def add_losing_matches(df):
    win_rename = {
        "WTeamID": "TeamIdA", 
        "WScore" : "ScoreA", 
        "LTeamID" : "TeamIdB",
        "LScore": "ScoreB",
     }
    win_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('W')})
    win_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('L')})
    
    lose_rename = {
        "WTeamID": "TeamIdB", 
        "WScore" : "ScoreB", 
        "LTeamID" : "TeamIdA",
        "LScore": "ScoreA",
    }
    lose_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('W')})
    lose_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('L')})
    
    win_df = df.copy()
    lose_df = df.copy()
    
    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)
    
    return pd.concat([win_df, lose_df], 0, sort=False)

In [65]:
df = add_losing_matches(df)

C:\Users\rcpat\AppData\Local\Temp\ipykernel_14896\135867894.py:26: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.



In [66]:
df.head()

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,WinRatioA,GapAvgA,538RatingA,WinRatio_L10A,WinRatio_SeasonL10_DiffA,KenPomA,SeedA,WinRatioB,GapAvgB,538RatingB,WinRatio_L10B,WinRatio_SeasonL10_DiffB,KenPomB,SeedB
0,2016,134,1195,96,1192,65,0.551724,2.724138,71.41,0.7,-0.148276,264,16.0,0.548387,-1.677419,66.72,0.6,-0.051613,321,16.0
1,2016,134,1455,70,1435,50,0.741935,13.709677,86.59,0.7,0.041935,44,11.0,0.593750,9.406250,85.59,0.6,-0.006250,38,11.0
2,2016,135,1221,59,1380,55,0.424242,-4.333333,66.85,0.5,-0.075758,324,16.0,0.612903,1.870968,67.96,0.6,0.012903,263,16.0
3,2016,135,1276,67,1409,62,0.636364,6.242424,79.57,0.5,0.136364,56,11.0,0.645161,4.322581,79.93,0.6,0.045161,78,11.0
4,2016,136,1114,85,1345,83,0.870968,9.935484,78.90,0.8,0.070968,213,12.0,0.764706,13.147059,88.68,0.7,0.064706,19,5.0


In [67]:
cols_to_diff = [
    'Seed', 'WinRatio', 'GapAvg', '538Rating', 'KenPom','WinRatio_L10','WinRatio_SeasonL10_Diff'
]

for col in cols_to_diff:
    df[col + 'Diff'] = df[col + 'A'] - df[col + 'B']
    
df.head()

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,WinRatioA,GapAvgA,538RatingA,WinRatio_L10A,WinRatio_SeasonL10_DiffA,KenPomA,SeedA,WinRatioB,GapAvgB,538RatingB,WinRatio_L10B,WinRatio_SeasonL10_DiffB,KenPomB,SeedB,SeedDiff,WinRatioDiff,GapAvgDiff,538RatingDiff,KenPomDiff,WinRatio_L10Diff,WinRatio_SeasonL10_DiffDiff
0,2016,134,1195,96,1192,65,0.551724,2.724138,71.41,0.7,-0.148276,264,16.0,0.548387,-1.677419,66.72,0.6,-0.051613,321,16.0,0.0,0.003337,4.401557,4.69,-57,0.1,-0.096663
1,2016,134,1455,70,1435,50,0.741935,13.709677,86.59,0.7,0.041935,44,11.0,0.593750,9.406250,85.59,0.6,-0.006250,38,11.0,0.0,0.148185,4.303427,1.00,6,0.1,0.048185
2,2016,135,1221,59,1380,55,0.424242,-4.333333,66.85,0.5,-0.075758,324,16.0,0.612903,1.870968,67.96,0.6,0.012903,263,16.0,0.0,-0.188661,-6.204301,-1.11,61,-0.1,-0.088661
3,2016,135,1276,67,1409,62,0.636364,6.242424,79.57,0.5,0.136364,56,11.0,0.645161,4.322581,79.93,0.6,0.045161,78,11.0,0.0,-0.008798,1.919844,-0.36,-22,-0.1,0.091202
4,2016,136,1114,85,1345,83,0.870968,9.935484,78.90,0.8,0.070968,213,12.0,0.764706,13.147059,88.68,0.7,0.064706,19,5.0,7.0,0.106262,-3.211575,-9.78,194,0.1,0.006262


In [68]:
df_test = pd.read_csv(INPUT_FOLDER + "MSampleSubmissionStage1.csv")

df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIdA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIdB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))

df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB
0,2016_1112_1114,0.5,2016,1112,1114
1,2016_1112_1122,0.5,2016,1112,1122
2,2016_1112_1124,0.5,2016,1112,1124
3,2016_1112_1138,0.5,2016,1112,1138
4,2016_1112_1139,0.5,2016,1112,1139


In [69]:
df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsA',
    'NumLosses': 'NumLossesA',
    'GapWins': 'GapWinsA',
    'GapLosses': 'GapLossesA',
    'WinRatio': 'WinRatioA',
    'GapAvg': 'GapAvgA',
    '538rating': '538RatingA',
    'WinRatio_L10': 'WinRatio_L10A',
    'WinRatio_SeasonL10_Diff': 'WinRatio_SeasonL10_DiffA',
    'KenPom': 'KenPomA',
    'Seed': 'SeedA'
}).drop(columns='TeamID', axis=1)

df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdB'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsB',
    'NumLosses': 'NumLossesB',
    'GapWins': 'GapWinsB',
    'GapLosses': 'GapLossesB',
    'WinRatio': 'WinRatioB',
    'GapAvg': 'GapAvgB',
    '538rating': '538RatingB',
    'WinRatio_L10': 'WinRatio_L10B',
    'WinRatio_SeasonL10_Diff': 'WinRatio_SeasonL10_DiffB',
    'KenPom': 'KenPomB',
    'Seed': 'SeedB'
}).drop(columns='TeamID', axis=1)

for col in cols_to_diff:
    df_test[col + 'Diff'] = df_test[col + 'A'] - df_test[col + 'B']

df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB,WinRatioA,GapAvgA,538RatingA,WinRatio_L10A,WinRatio_SeasonL10_DiffA,KenPomA,SeedA,WinRatioB,GapAvgB,538RatingB,WinRatio_L10B,WinRatio_SeasonL10_DiffB,KenPomB,SeedB,SeedDiff,WinRatioDiff,GapAvgDiff,538RatingDiff,KenPomDiff,WinRatio_L10Diff,WinRatio_SeasonL10_DiffDiff
0,2016_1112_1114,0.5,2016,1112,1114,0.757576,12.212121,89.04,0.7,0.057576,19,6.0,0.870968,9.935484,78.90,0.8,0.070968,213,12.0,-6.0,-0.113392,2.276637,10.14,-194,-0.1,-0.013392
1,2016_1112_1122,0.5,2016,1112,1122,0.757576,12.212121,89.04,0.7,0.057576,19,6.0,0.484848,-2.363636,68.83,0.8,-0.315152,287,16.0,-10.0,0.272727,14.575758,20.21,-268,-0.1,0.372727
2,2016_1112_1124,0.5,2016,1112,1124,0.757576,12.212121,89.04,0.7,0.057576,19,6.0,0.656250,6.687500,85.47,0.5,0.156250,38,5.0,1.0,0.101326,5.524621,3.57,-19,0.2,-0.098674
3,2016_1112_1138,0.5,2016,1112,1138,0.757576,12.212121,89.04,0.7,0.057576,19,6.0,0.575758,0.666667,75.66,0.6,-0.024242,179,14.0,-8.0,0.181818,11.545455,13.38,-160,0.1,0.081818
4,2016_1112_1139,0.5,2016,1112,1139,0.757576,12.212121,89.04,0.7,0.057576,19,6.0,0.677419,9.419355,84.17,0.7,-0.022581,48,9.0,-3.0,0.080156,2.792766,4.87,-29,0.0,0.080156


In [70]:
df['ScoreDiff'] = df['ScoreA'] - df['ScoreB']
df['WinA'] = (df['ScoreDiff'] > 0).astype(int)

In [71]:
features = [
    'SeedDiff',
    '538RatingDiff',
    'WinRatioDiff', 
    'GapAvgDiff',
    'KenPomDiff',
    'WinRatio_L10Diff',
    'WinRatio_SeasonL10_DiffDiff',
]

In [72]:
submission = pd.read_csv(INPUT_FOLDER + 'SampleSubmission2023.csv')
submission['Season'] = submission['ID'].apply(lambda x: int(x.split('_')[0]))
submission['TeamIdA'] = submission['ID'].apply(lambda x: int(x.split('_')[1]))
submission['TeamIdB'] = submission['ID'].apply(lambda x: int(x.split('_')[2]))

submission = pd.merge(
    submission,
    df_features_season,
    how='inner',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsA',
    'NumLosses': 'NumLossesA',
    'GapWins': 'GapWinsA',
    'GapLosses': 'GapLossesA',
    'WinRatio': 'WinRatioA',
    'GapAvg': 'GapAvgA',
    '538rating': '538RatingA',
    'WinRatio_L10': 'WinRatio_L10A',
    'WinRatio_SeasonL10_Diff': 'WinRatio_SeasonL10_DiffA',
    'KenPom': 'KenPomA',
    'Seed': 'SeedA'
}).drop(columns='TeamID', axis=1)

submission = pd.merge(
    submission,
    df_features_season,
    how='inner',
    left_on=['Season', 'TeamIdB'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsB',
    'NumLosses': 'NumLossesB',
    'GapWins': 'GapWinsB',
    'GapLosses': 'GapLossesB',
    'WinRatio': 'WinRatioB',
    'GapAvg': 'GapAvgB',
    '538rating': '538RatingB',
    'WinRatio_L10': 'WinRatio_L10B',
    'WinRatio_SeasonL10_Diff': 'WinRatio_SeasonL10_DiffB',
    'KenPom': 'KenPomB',
    'Seed': 'SeedB'
}).drop(columns='TeamID', axis=1)

for col in cols_to_diff:
    submission[col + 'Diff'] = submission[col + 'A'] - submission[col + 'B']

submission.head()

,ID,Pred,Season,TeamIdA,TeamIdB,WinRatioA,GapAvgA,538RatingA,WinRatio_L10A,WinRatio_SeasonL10_DiffA,KenPomA,SeedA,WinRatioB,GapAvgB,538RatingB,WinRatio_L10B,WinRatio_SeasonL10_DiffB,KenPomB,SeedB,SeedDiff,WinRatioDiff,GapAvgDiff,538RatingDiff,KenPomDiff,WinRatio_L10Diff,WinRatio_SeasonL10_DiffDiff
0,2023_1104_1112,0.5,2023,1104,1112,0.852941,13.676471,92.24,0.8,0.052941,15,1.0,0.823529,11.176471,88.98,0.7,0.123529,20,2.0,-1.0,0.029412,2.500000,3.26,-5,0.1,-0.070588
1,2023_1104_1113,0.5,2023,1104,1113,0.852941,13.676471,92.24,0.8,0.052941,15,1.0,0.647059,2.617647,80.36,0.6,0.047059,77,11.0,-10.0,0.205882,11.058824,11.88,-62,0.2,0.005882
2,2023_1112_1113,0.5,2023,1112,1113,0.823529,11.176471,88.98,0.7,0.123529,20,2.0,0.647059,2.617647,80.36,0.6,0.047059,77,11.0,-9.0,0.176471,8.558824,8.62,-57,0.1,0.076471
3,2023_1104_1116,0.5,2023,1104,1116,0.852941,13.676471,92.24,0.8,0.052941,15,1.0,0.606061,7.030303,85.12,0.4,0.206061,27,8.0,-7.0,0.246881,6.646168,7.12,-12,0.4,-0.153119
4,2023_1112_1116,0.5,2023,1112,1116,0.823529,11.176471,88.98,0.7,0.123529,20,2.0,0.606061,7.030303,85.12,0.4,0.206061,27,8.0,-6.0,0.217469,4.146168,3.86,-7,0.3,-0.082531


In [83]:
# Sample and Get Target
x = df[df['Season'] < 2022]
valid = df[df['Season'] == 2022]

x_train = x[features]
y_train = x['WinA']

x_valid = valid[features]
y_valid = valid['WinA']

sub_x = submission[features]

In [74]:
# Scale and Polynomial
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_trans = ss.fit_transform(x_train)
x_train = pd.DataFrame(x_trans, columns = x_train.columns)

ss.fit(x_valid)
x_trans = ss.fit_transform(x_valid)
x_valid = pd.DataFrame(x_trans, columns = x_valid.columns)

ss.fit(sub_x)
x_trans = ss.fit_transform(sub_x)
sub_x = pd.DataFrame(x_trans, columns = sub_x.columns)


In [84]:
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [85]:
ebm = ExplainableBoostingClassifier(random_state=2023)
ebm.fit(x_train, y_train)

ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7001/1938909195344/ -->

In [86]:
print(ebm.score(x_train,y_train))
print(ebm.score(x_valid,y_valid))

0.7784431137724551
0.6716417910447762


In [78]:
def Extract(lst):
    return [item[1] for item in lst]

preds = Extract(ebm.predict_proba(sub_x))

In [79]:
d = {'ID':submission.iloc[:,0],'Pred':preds}
preds_df = pd.DataFrame.from_dict(d)

In [66]:
preds_df.to_csv('SubmissionMens2023.csv')

In [67]:
preds_df.shape

(2278, 2)

In [80]:
preds_df['Season'] = preds_df['ID'].apply(lambda x: int(x.split('_')[0]))
preds_df['TeamIdA'] = preds_df['ID'].apply(lambda x: int(x.split('_')[1]))
preds_df['TeamIdB'] = preds_df['ID'].apply(lambda x: int(x.split('_')[2]))

preds_df.head()

,ID,Pred,Season,TeamIdA,TeamIdB
0,2023_1104_1112,0.579088,2023,1104,1112
1,2023_1104_1113,0.862637,2023,1104,1113
2,2023_1112_1113,0.810393,2023,1112,1113
3,2023_1104_1116,0.749117,2023,1104,1116
4,2023_1112_1116,0.626897,2023,1112,1116


In [81]:
preds_df = pd.merge(preds_df,teams, how = 'left', left_on = 'TeamIdA', right_on = 'TeamID').rename(columns = {'TeamName': 'Name_A'})
preds_df = pd.merge(preds_df,teams, how = 'left', left_on = 'TeamIdB', right_on = 'TeamID').rename(columns = {'TeamName': 'Name_B'})

In [82]:
preds_df[['Name_A','Name_B','Pred']].to_csv('BracketModel.csv')